In [1]:
import nltk
import string
import numpy as np
import pandas as pd

from nltk.corpus import stopwords
from nltk.util import ngrams
from sklearn.feature_extraction.text import CountVectorizer
from collections import defaultdict, Counter

import re
from nltk.tokenize import word_tokenize
import gensim
import string
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tqdm import tqdm

import seaborn as sns
import matplotlib.pyplot as plt

### Loading & viewing dataset.
- References [link_1](https://github.com/Nhan121/Kaggle-6-first-projects/blob/master/NLP_Text_Classification/NLP_Text_classification.ipynb)

In [2]:
path = r'../input/nlp-getting-started'
train = pd.read_csv(path + '/train.csv')
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


## 1. Pre-processing
### 1.1. Counting & handling missing-values

In [3]:
pd.DataFrame({'count_NA': train.isna().sum(), 
              '%NA': round(100*train.isna().sum() / train.shape[0], 2), 
              'type' :train.dtypes})

,count_NA,%NA,type
id,0,0.00,int64
keyword,61,0.80,object
location,2533,33.27,object
text,0,0.00,object
target,0,0.00,int64


#### Comment.
- For the column `keyword`, it takes about `0.8%` at both dataset; whilethe column `location` is more than `33%`. Both of them (both columns) are the `object` types so we can replace the missing values by `unknown` to keep the structure preservation.
- The rest columns `id, text, target` (in `train.csv`) and `id, text` (in `test.csv`) has no missing value since we will predict the `target` (and add them to the `test`) mainly based on the `text`.

In [4]:
train = train.fillna('unknown')
train.head()

,id,keyword,location,text,target
0,1,unknown,unknown,Our Deeds are the Reason of this #earthquake M...,1
1,4,unknown,unknown,Forest fire near La Ronge Sask. Canada,1
2,5,unknown,unknown,All residents asked to 'shelter in place' are ...,1
3,6,unknown,unknown,"13,000 people receive #wildfires evacuation or...",1
4,7,unknown,unknown,Just got sent this photo from Ruby #Alaska as ...,1


### 1.2. Droping duplicated-values
#### The unique-values.

In [5]:
count_unique = [len(train[col].unique()) for col in train.columns]
percent_uniq = [round(100*cnt / train.shape[0], 2) for cnt in count_unique]
pd.DataFrame({'cnt_uniq': count_unique, 'perc_uniq_%': percent_uniq}, index = train.columns)

,cnt_uniq,perc_uniq_%
id,7613,100.00
keyword,222,2.92
location,3342,43.90
text,7503,98.56
target,2,0.03


#### Droping the duplicates values (after ignore the `id`)

In [6]:
train_non_id = train.drop(columns = ['id'])
train_non_id = train_non_id.drop_duplicates()
train_non_id.shape

(7561, 4)

### 1.3. Text-processing.

In [7]:
!pip install pyspellchecker

     |████████████████████████████████| 2.5 MB 2.1 MB/s eta 0:00:01     |██                              | 153 kB 414 kB/s eta 0:00:06
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


The following function is prepared for the Section 3.2 (Model using `text-processing`)

In [8]:
import re
from spellchecker import SpellChecker    

def process_text(str_input):
    ## 1. Remove url_link
    remove_url = re.compile(r'https?://\S+|www\.\S+').sub(r'', str_input)
    
    ## 2. Remove html_link
    remove_html = re.compile(r'<.*?>').sub(r'', remove_url)
    
    ## 3. Remove Emojis
    remove_emo = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE).sub(r'', remove_html)
    words = re.sub(r"[^A-Za-z0-9\-]", " ", remove_emo).lower().split()    
        
    ## 4. spell_correction
    #spell = SpellChecker()
    #words = [spell.correction(word) for word in words[:50]]

    return words

## Review the first 5 lines after using text-processing
new_train = train.copy()
%time new_train.loc[:5, 'correct_text'] = train.loc[:5, 'text'].apply(lambda x: process_text(x))
new_train.head()

CPU times: user 9.16 ms, sys: 754 µs, total: 9.91 ms
Wall time: 9.56 ms


,id,keyword,location,text,target,correct_text
0,1,unknown,unknown,Our Deeds are the Reason of this #earthquake M...,1,"[our, deeds, are, the, reason, of, this, earth..."
1,4,unknown,unknown,Forest fire near La Ronge Sask. Canada,1,"[forest, fire, near, la, ronge, sask, canada]"
2,5,unknown,unknown,All residents asked to 'shelter in place' are ...,1,"[all, residents, asked, to, shelter, in, place..."
3,6,unknown,unknown,"13,000 people receive #wildfires evacuation or...",1,"[13, 000, people, receive, wildfires, evacuati..."
4,7,unknown,unknown,Just got sent this photo from Ruby #Alaska as ...,1,"[just, got, sent, this, photo, from, ruby, ala..."


## 2. Grid-Seach CV.
### 2.1. Initialize the model

In [9]:
train['Text_length'] = train['text'].str.len()
train['Numb_words'] = train['text'].str.split().map(lambda x: len(x))
train = train.set_index('id')
train.head()

,keyword,location,text,target,Text_length,Numb_words
id,,,,,,
1,unknown,unknown,Our Deeds are the Reason of this #earthquake M...,1,69,13
4,unknown,unknown,Forest fire near La Ronge Sask. Canada,1,38,7
5,unknown,unknown,All residents asked to 'shelter in place' are ...,1,133,22
6,unknown,unknown,"13,000 people receive #wildfires evacuation or...",1,65,8
7,unknown,unknown,Just got sent this photo from Ruby #Alaska as ...,1,88,16


### Train-test split.
- 2 columns `keyword, location` has so many categories so that it can not be used to establish the model.
- It make more time in computation and the efficients is not good.

In [10]:
# Initialize the tfidf_vectorizer 
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words = 'english') 
X = tfidf_vectorizer.fit_transform(train['text']) 

## Target
y = train['target']

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y, test_size=0.3, random_state=42)

X_train.shape, X_test.shape, y_train.shape

((5329, 21363), (2284, 21363), (5329,))

### 2.2. Lauching with SVM model

In [12]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV

svm = svm.SVC()
grid_params = [{
                'kernel':['linear', 'rbf', 'poly'],
                'C': [0.1, 1, 5], #default: 1.0
                 }]
clf = GridSearchCV(estimator=svm, param_grid = grid_params, cv = 8, verbose = 0)
%time clf.fit(X_train, y_train)

CPU times: user 3min 38s, sys: 4.04 s, total: 3min 42s
Wall time: 3min 42s


GridSearchCV(cv=8, estimator=SVC(),
             param_grid=[{'C': [0.1, 1, 5],
                          'kernel': ['linear', 'rbf', 'poly']}])

In [13]:
clf.best_params_

{'C': 1, 'kernel': 'linear'}

In [14]:
clf.best_score_

0.7907679043361202

In [15]:
from sklearn.metrics import accuracy_score

pred_train = clf.predict(X_train)
pred_test = clf.predict(X_test)

print(accuracy_score(y_train, pred_train))
print(accuracy_score(y_test, pred_test))

0.9502720960780634
0.8104203152364273


In [16]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_train, pred_train))
print(confusion_matrix(y_test, pred_test))

[[2994   45]
 [ 220 2070]]
[[1178  125]
 [ 308  673]]


In [17]:
df = pd.DataFrame(columns = ['best_params', 'train_acc_%', 'train_conf_matrix', 'test_acc_%', 'test_conf_matrix'])
df.loc['SVM'] = [clf.best_params_, 
                 100*accuracy_score(y_train, pred_train), confusion_matrix(y_train, pred_train), 
                 100*accuracy_score(y_test, pred_test), confusion_matrix(y_test, pred_test)]
df

,best_params,train_acc_%,train_conf_matrix,test_acc_%,test_conf_matrix
SVM,"{'C': 1, 'kernel': 'linear'}",95.02721,"[[2994, 45], [220, 2070]]",81.042032,"[[1178, 125], [308, 673]]"


## 3. And for another model

In [18]:
from sklearn.preprocessing import StandardScaler
X_train_num = train[['Text_length', 'Numb_words']].to_numpy()
X_con = StandardScaler().fit_transform(X_train_num)

from sklearn.feature_extraction.text import CountVectorizer
X_C = CountVectorizer(analyzer = process_text).fit_transform(train['text'])

X_train, X_test, y_train, y_test = train_test_split(X_C, y, stratify = y, test_size=0.3, random_state=0)

X_train.shape, X_test.shape, y_train.shape

((5329, 17330), (2284, 17330), (5329,))

### 3.1. Naive Bayes

In [19]:
from sklearn.naive_bayes import MultinomialNB, GaussianNB

naiv = MultinomialNB()
grid_params = [{'alpha' : [0.5, 0.75, 0.8, 1]}]
clf = GridSearchCV(estimator=naiv, param_grid = grid_params, cv = 5, verbose = 0)
%time clf.fit(X_train, y_train)

pred_train = clf.predict(X_train)
pred_test = clf.predict(X_test)

print(clf.best_params_)
print(accuracy_score(y_train, pred_train))
print(accuracy_score(y_test, pred_test))

df.loc['MultiNB'] = [clf.best_params_, 
                     100*accuracy_score(y_train, pred_train), confusion_matrix(y_train, pred_train), 
                     100*accuracy_score(y_test, pred_test), confusion_matrix(y_test, pred_test)]

CPU times: user 124 ms, sys: 991 µs, total: 125 ms
Wall time: 124 ms
{'alpha': 1}
0.903921936573466
0.8025394045534151


### 3.2. Logistic-Regression

In [20]:
from sklearn.linear_model import LogisticRegression

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y, test_size=0.3, random_state=42)
logreg = LogisticRegression()
grid_params = [{'C' : [0.5, 1, 2, 10],
                'max_iter': [500, 1000]}]
clf = GridSearchCV(estimator = logreg, param_grid = grid_params, cv = 15, verbose = 0)
%time clf.fit(X_train, y_train)

pred_train = clf.predict(X_train)
pred_test = clf.predict(X_test)

print(clf.best_params_)
print(accuracy_score(y_train, pred_train))
print(accuracy_score(y_test, pred_test))

df.loc['Log-reg'] = [clf.best_params_, 
                     100*accuracy_score(y_train, pred_train), confusion_matrix(y_train, pred_train), 
                     100*accuracy_score(y_test, pred_test), confusion_matrix(y_test, pred_test)]

CPU times: user 45 s, sys: 630 ms, total: 45.6 s
Wall time: 23 s
{'C': 2, 'max_iter': 500}
0.9356352036029274
0.8042907180385289


### 3.3. K-Nearest-Neighbors Classifier

In [21]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
grid_params = [{'n_neighbors': [50, 80, 100, 150]}]

clf = GridSearchCV(estimator = knn, param_grid = grid_params, cv = 5, verbose = 0)
%time clf.fit(X_train, y_train)

pred_train = clf.predict(X_train)
pred_test = clf.predict(X_test)

print(clf.best_params_)
print(accuracy_score(y_train, pred_train))
print(accuracy_score(y_test, pred_test))

df.loc['K-nn'] = [clf.best_params_, 
                  100*accuracy_score(y_train, pred_train), confusion_matrix(y_train, pred_train), 
                  100*accuracy_score(y_test, pred_test), confusion_matrix(y_test, pred_test)]

CPU times: user 3.07 s, sys: 143 ms, total: 3.22 s
Wall time: 3.17 s
{'n_neighbors': 100}
0.7766935635203603
0.7863397548161121


### 3.4. Random Forest Classifier

In [22]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier()
X_train, X_test, y_train, y_test = train_test_split(X_C, y, stratify = y, test_size=0.3, random_state=42)
grid_params = [{
                'n_estimators': [200, 300], #default=10
                'criterion': ['gini'], #default=”gini”
                'max_depth': [5, 10, 20], #default=None
                'oob_score': [True], #default=False
                'random_state': [0, 42, 88]
                 }]

clf = GridSearchCV(estimator = rfc, param_grid = grid_params, cv = 5, verbose = 0)
%time clf.fit(X_train, y_train)

pred_train = clf.predict(X_train)
pred_test = clf.predict(X_test)

print(clf.best_params_)
print(accuracy_score(y_train, pred_train))
print(accuracy_score(y_test, pred_test))

df.loc['rfc'] = [clf.best_params_, 
                 100*accuracy_score(y_train, pred_train), confusion_matrix(y_train, pred_train), 
                 100*accuracy_score(y_test, pred_test), confusion_matrix(y_test, pred_test)]

CPU times: user 3min 46s, sys: 666 ms, total: 3min 47s
Wall time: 3min 47s
{'criterion': 'gini', 'max_depth': 20, 'n_estimators': 300, 'oob_score': True, 'random_state': 88}
0.7350347157065116
0.717600700525394


### 3.5. XGB Classifier.

In [23]:
from xgboost import XGBClassifier

xgb = XGBClassifier()
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y, test_size=0.3, random_state=42)
grid_params = [{
                'n_estimators': [500, 1000, 5000],
                'learning_rate': [.05, 0.1],
                'max_depth': [5, 7, 10],
            }]
clf = GridSearchCV(estimator = xgb, param_grid = grid_params, cv = 5, verbose = 0)
%time clf.fit(X_train, y_train,eval_set=[(X_train, y_train), (X_test, y_test)], early_stopping_rounds = 20, verbose = 0)

CPU times: user 47min 44s, sys: 1min 40s, total: 49min 24s
Wall time: 12min 42s


GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, gamma=None,
                                     gpu_id=None, importance_type='gain',
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=None,
                                     num_parallel_tree=None, random_state=None,
                                     reg_alpha=None, reg_lambda=None,
                                     scale_pos_weight=None, subsample=None,
                                     tree_method=None,

In [24]:
print(clf.best_params_)
print(accuracy_score(y_train, pred_train))
print(accuracy_score(y_test, pred_test))

df.loc['XGB-simple'] = [clf.best_params_, 
                         100*accuracy_score(y_train, pred_train), confusion_matrix(y_train, pred_train), 
                         100*accuracy_score(y_test, pred_test), confusion_matrix(y_test, pred_test)]

{'learning_rate': 0.1, 'max_depth': 10, 'n_estimators': 500}
0.7350347157065116
0.717600700525394


#### XGB with complexity-model

In [25]:
from sklearn.base import BaseEstimator, TransformerMixin

class TextSelector(BaseEstimator, TransformerMixin):
    def __init__(self, field):
        self.field = field
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.field]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    def __init__(self, field):
        self.field = field
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[[self.field]]

In [26]:
import time
from sklearn.svm import SVC
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import Pipeline, FeatureUnion
svc = SVC()

X_t = train[['text', 'Text_length']]
X_train, X_test, y_train, y_test = train_test_split(X_t, 
                                                    y, 
                                                    test_size = 0.3, 
                                                    stratify = y, 
                                                    random_state = 42)
clf = Pipeline([
    (
        'features', FeatureUnion([
        ('text', Pipeline([
            ('colext', TextSelector('text')),
            ('tfidf', TfidfVectorizer(tokenizer = process_text, stop_words = 'english',
                     min_df = .0025, max_df = 0.25, ngram_range = (1, 5) ) ),
            ('svd', TruncatedSVD(algorithm ='randomized', n_components = 300) ), #for XGB
        ])),
        ('words', Pipeline([
            ('wordext', NumberSelector('Text_length')),
            ('wscaler', StandardScaler()),
        ])),            
    ])
    ),
    ('clf', XGBClassifier(eval_set=[(X_train, y_train), (X_test, y_test)],
                          max_depth = 8, n_estimators = 500, base_estimator = svc, learning_rate = 0.1, cv = 15))
    ])

## Fit the model
start = time.time()
clf.fit(X_train, y_train)
pred_train = clf.predict(X_train)
pred_test = clf.predict(X_test)
print ('Fit&trainning time : ', time.time() - start)

train_acc_Xgb2 = accuracy_score(y_train, clf.predict(X_train)) * 100.0 
test_acc_Xgb2 = accuracy_score(y_test, pred_test) * 100.0

print("Training_Accuracy: %.2f%%" % train_acc_Xgb2)
print("Testing_Accuracy: %.2f%%" % test_acc_Xgb2)

[09:39:48] WARNING: ../src/learner.cc:516: 
Parameters: { base_estimator, cv, eval_set } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


Fit&trainning time :  66.12007093429565
Training_Accuracy: 99.02%
Testing_Accuracy: 77.01%


In [27]:
df.loc['XGB-complex'] = ["", 
                         100*accuracy_score(y_train, pred_train), confusion_matrix(y_train, pred_train), 
                         100*accuracy_score(y_test, pred_test), confusion_matrix(y_test, pred_test)]

### 3.6. Adaboost

In [28]:
from sklearn.ensemble import AdaBoostClassifier

X_t = train[['text', 'Numb_words']]
X_train, X_test, y_train, y_test = train_test_split(X_t, 
                                                    y, 
                                                    test_size = 0.3, 
                                                    stratify = y, 
                                                    random_state = 42)

clf = Pipeline([
    (
        'features', FeatureUnion([
        ('text', Pipeline([
            ('colext', TextSelector('text')),
            ('tfidf', TfidfVectorizer(tokenizer = process_text, stop_words = 'english',
                     min_df = .0025, max_df = 0.25, ngram_range = (1, 5) ) ),
            ('svd', TruncatedSVD(algorithm ='randomized', n_components = 300) ), 
        ])),
        ('words', Pipeline([
            ('wordext', NumberSelector('Numb_words')),
            ('wscaler', StandardScaler()),
        ])),            
    ])
    ),
    ('clf', AdaBoostClassifier(n_estimators = 300, learning_rate = 0.1)),
    ])

start = time.time()
%time clf.fit(X_train, y_train)
pred_train = clf.predict(X_train)
pred_test = clf.predict(X_test)

df.loc['Ada_Bst-complex'] = ["", 
                             100*accuracy_score(y_train, pred_train), confusion_matrix(y_train, pred_train), 
                             100*accuracy_score(y_test, pred_test), confusion_matrix(y_test, pred_test)]
df

CPU times: user 1min 9s, sys: 56.4 ms, total: 1min 9s
Wall time: 1min 9s


,best_params,train_acc_%,train_conf_matrix,test_acc_%,test_conf_matrix
SVM,"{'C': 1, 'kernel': 'linear'}",95.027210,"[[2994, 45], [220, 2070]]",81.042032,"[[1178, 125], [308, 673]]"
MultiNB,{'alpha': 1},90.392194,"[[2917, 122], [390, 1900]]",80.253940,"[[1122, 181], [270, 711]]"
Log-reg,"{'C': 2, 'max_iter': 500}",93.563520,"[[2991, 48], [295, 1995]]",80.429072,"[[1180, 123], [324, 657]]"
K-nn,{'n_neighbors': 100},77.669356,"[[2649, 390], [800, 1490]]",78.633975,"[[1122, 181], [307, 674]]"
rfc,"{'criterion': 'gini', 'max_depth': 20, 'n_esti...",73.503472,"[[3039, 0], [1412, 878]]",71.760070,"[[1292, 11], [634, 347]]"
XGB-simple,"{'learning_rate': 0.1, 'max_depth': 10, 'n_est...",73.503472,"[[3039, 0], [1412, 878]]",71.760070,"[[1292, 11], [634, 347]]"
XGB-complex,,99.024207,"[[3026, 13], [39, 2251]]",77.014011,"[[1115, 188], [337, 644]]"
Ada_Bst-complex,,77.575530,"[[2690, 349], [846, 1444]]",75.043783,"[[1120, 183], [387, 594]]"
